# Введение



Сегодня мы продолжим изучение применения методов анализа данных и машинного обучения на практических примерах. В прошлой ЛР мы с вами разбирались с задачей кластеризации. Теперь вас ждет новая работа - «Задачи о паспортах» (Задание №2).
При решении будут показаны основы анализа текстовой информации, а также ее кодирование для построения модели с помощью Python и модулей для анализа данных (pandas, scikit-learn, pymorphy).

Документация
* [pandas](https://pandas.pydata.org/pandas-docs/stable/)
* [scikit-learn](https://scikit-learn.org/stable/)
* [pymorphy](https://pymorphy2.readthedocs.io/en/latest/index.html)

Постановка задачи: 

При работе с большим объёмом данных важно поддерживать их чистоту. А при заполнении заявки на банковский продукт необходимо указывать полные паспортные данные, в том числе и поле «кем выдан паспорт», число различных вариантов написаний одного и того же отделения потенциальными клиентами может достигать нескольких сотен. Важно понимать, не ошибся ли клиент, заполняя другие поля: «код подразделения», «серию/номер паспорта». Для этого необходимо сверять «код подразделения» и «кем выдан паспорт».
Задача заключается в том, чтобы проставить коды подразделений для записей из тестовой выборки, основываясь на обучающей выборке.

 Загрузим данные и все библиотеки, которые нам пригодятся. 
 
 Для полного понимания и дополнительной информации вы можете обращаться к документации



In [ ]:
pip install pymorphy2

     |████████████████████████████████| 61kB 3.3MB/s 
     |████████████████████████████████| 8.2MB 6.0MB/s 


In [ ]:
from pandas import read_csv
import pymorphy2
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.decomposition import PCA

Давайте посмотрим, как выглядят наши 

---

данные:

In [ ]:
train = read_csv('passport_training_set.csv',';', index_col='id' ,encoding='cp1251')
train.shape

(96750, 3)

In [ ]:
train.head(5)

,passport_div_code,passport_issuer_name,passport_issue_month/year
id,,,
1,422008,БЕЛОВСКИМ УВД КЕМЕРОВСКОЙ ОБЛАСТИ,11M2001
2,500112,ТП №2 В ГОР. ОРЕХОВО-ЗУЕВО ОУФМС РОССИИ ПО МО ...,03M2009
3,642001,ВОЛЖСКИМ РОВД ГОР.САРАТОВА,04M2002
4,162004,УВД МОСКОВСКОГО РАЙОНА Г.КАЗАНЬ,12M2002
5,80001,ОТДЕЛОМ ОФМС РОССИИ ПО РЕСП КАЛМЫКИЯ В Г ЭЛИСТА,08M2009


Мы с вами видим таблицу, состоящую из 3 колонок - Кода подразделения, Кем выдан паспорт и дата выдачи, в формате mm/yyyy. На данных из колонок passport_issuer_name и passport_issue_month/year мы будем тренировать наши данные. Данные из колонки passport_div_code - наш таргет. Именно код подразделения мы будем предсказывать.

Т.е. на passport_issuer_name(Кем выдан) и passport_issue_month/year(Дата выдачи) мы должны тренировать наши данные, а предсказывать будем passport_div_code (Код подразделения)


Как вы можете заметить, данные в колонке passport_issuer_name представлены не в числовом, уже знакомым для нас, формате, а в текстовом. А это значит что нам предстоит привести текст в нормальную форму. 

















### Предварительная обработка данных

Теперь можно посмотреть как пользователи записывают поле «кем выдан паспорт» на примере какого-либо подразделения:

In [ ]:
example_code = train.passport_div_code[train.passport_div_code.duplicated()].values[0]
for i in train.passport_issuer_name[train.passport_div_code == example_code].drop_duplicates():
    print (i)

ОТДЕЛЕНИЕМ УФМС РОССИИ ПО РЕСПУБЛИКЕ КАРЕЛИЯ В МЕДВЕЖ. Р-Е
ОТДЕЛЕНИЕМ УФМС РОССИИ ПО Р. КАРЕЛИЯ В МЕДВЕЖЬЕГОРСКОМ РАЙОНЕ
ОТДЕЛЕНИЕМ УФМС РОССИИ ПО РЕСП КАРЕЛИЯ В МЕДВЕЖЬЕГОРСКОМ Р-НЕ
ОТДЕЛЕНИЕМ УФМС РОССИИ ПО РЕСПУБЛИКЕ КАРЕЛИЯ В МЕДВЕЖЬЕГОРСКОМ РАЙОНЕ
ОУФМС РОССИИ ПО РЕСПУБЛИКЕ КАРЕЛИЯ В МЕДВЕЖЬЕГОРСКОМ РАЙОНЕ
УФМС РОССИИ ПО РК В МЕДВЕЖЬЕГОРСКОМ РАЙОНЕ
ОТДЕЛЕНИЕМ УФМС РОССИИ ПО РЕСПУБЛИКЕ КАРЕЛИЯ МЕДВЕЖЬЕГОРСКОМ Р-ОНЕ
ОТДЕЛЕНИЕМ УФМС РОССИИ ПО РК В МЕДВЕЖЬЕГОРСКОМ РАЙОНЕ
ОТДЕЛЕНИЕМ УФМС РОССИИ ПО РЕСПУБЛИКЕ КОРЕЛИЯ В МЕДВЕЖИГОРСКОМ РАЙОНЕ
УФМС РОССИИ ПО Р. КАРЕЛИЯ МЕДВЕЖЬЕГОРСКОГО Р-НА
ОТДЕЛОМ УФМС РОССИИ ПО РЕСПУБЛИКЕ КАРЕЛИЯ В МЕДВЕЖЬЕГОРСКОМ
УФМС РЕСПУБЛИКИ КАРЕЛИИ МЕДВЕЖЬЕГОРСКОГО Р-ОН
МЕДВЕЖЬЕГОРСКИМ ОВД


Как можно заметить нужное нам поле действительно заполняется криво. Но для нормально кодирования мы должны привести это поле к более-менее нормальному (однозначному) виду.
Для начала вам предлагается привести все записи к одному регистру, например, чтобы все буквы стали строчными. Это легко сделать с помощью атрибута str, столбца DataFrame'a. Этот атрибут позволяет работать со столбцом как с строкой, а также выполнять различного рода поиск и замену по регулярным выражениям:

In [ ]:
train.passport_issuer_name = train.passport_issuer_name.str.lower()
train[train.passport_div_code == example_code].head(5)

,passport_div_code,passport_issuer_name,passport_issue_month/year
id,,,
19,100010,отделением уфмс россии по республике карелия в...,04M2008
22,100010,отделением уфмс россии по р. карелия в медвежь...,10M2009
5642,100010,отделением уфмс россии по респ карелия в медве...,08M2008
6668,100010,отделением уфмс россии по республике карелия в...,08M2011
8732,100010,отделением уфмс россии по республике карелия в...,08M2012


C регистром определились. 

####[Регулярные](https://tproger.ru/translations/regular-expression-python/) выражения

Говоря простым языком, регулярное выражение — это последовательность символов, используемая для поиска и замены текста в строке или файле.

Регулярные выражения используют два типа символов:

* специальные символы: как следует из названия, у этих символов есть специальные значения. Аналогично символу *, который как правило означает «любой символ» (но в регулярных выражениях работает немного иначе, о чем поговорим ниже);
* литералы (например: a, b, 1, 2 и т. д.).

В Python для работы с регулярными выражениями есть модуль re. Для использования его нужно импортировать:

In [ ]:
import re

Операторы и их описание

.	Один любой символ, кроме новой строки \n.

\w	Любая цифра или буква (\W — все, кроме буквы или цифры)

\d	Любая цифра [0-9] (\D — все, кроме цифры)

\s	Любой пробельный символ (\S — любой непробельный символ)

[..]	Один из символов в скобках ([^..] — любой символ, кроме тех, что в 
скобках)

\	Экранирование специальных символов (\. означает точку или \+ — знак «плюс»)

a|b	Соответствует a или b

()	Группирует выражение и возвращает найденный текст

\t, \n, \r	Символ табуляции, новой строки и возврата каретки соответственно

Попробуйте с помощью функции re.findall() найти:

* судьба, судоходство, судьбы, сударыня в тексте: "Анатолий любил заниматься судоходством и верил, что это его судьба! Но ирония судьбы в том, что сударыня Екатерина так не думала" ([а-я] - любая буква. [а-я]* - любые буквы дальше)
* дворе, двора в тексте: "На дворе трава, на траве дрова Не руби дрова на траве двора."
* покупки, покупочки в тексте: "Расскажите про покупки, Про какие про покупки? Про покупки, про покупки, Про покупочки мои."
* чертили, чертенка, чертеж,: "Четыре черненьких, чумазеньких чертенка Чертили черными чернилами чертеж." 

In [ ]:
re.findall('(по\куп[а-я]*|с\куп[а-я]*)', 'Мы любим наших покупателей. Рады когда они покупают что-то. А еще у нас лучшие скупщики') #покупателей, покупают, скупщики

['покупателей', 'покупают', 'скупщики']

In [ ]:
re.findall('суд\w*', 'Анатолий любил заниматься судоходством и верил, что это его судьба! Но ирония судьбы в том, что сударыня Екатерина так не думала') ## - ВАШ КОД ТУТ - ##

['судоходством', 'судьба', 'судьбы', 'сударыня']

In [ ]:
re.findall('двор[е|а]', 'На дворе трава, на траве дрова Не руби дрова на траве двора.') ## - ВАШ КОД ТУТ - ##

['дворе', 'двора']

In [ ]:
re.findall('(покуп[(ки)|(очки)])', 'Расскажите про покупки, Про какие про покупки? Про покупки, про покупки, Про покупочки мои.') ## - ВАШ КОД ТУТ - ##

['покупк', 'покупк', 'покупк', 'покупк', 'покупо']

In [ ]:
re.findall('[Ч|ч]ерт\w*', 'Четыре черненьких, чумазеньких чертенка Чертили черными чернилами чертеж.') ## - ВАШ КОД ТУТ - ##

['чертенка', 'Чертили', 'чертеж']

##### Сокращения

Далее надо по возможности избавиться от популярных сокращений, например район, город и т.д. Сделаем это с помощью регулярных выражений. Pandas предоставляет удобное использование регулярных выражений применительно к каждому столбцу. Это выглядит так: 


In [ ]:
result = re.search(r'р-(а|й|о|н|е)*', u'отделением уфмс россии по р. карелия в медвежь р-не') # Поиск в строке любых сокращений типа р- (а ИЛИ й ИЛИ о ИЛИ н ИЛИ е)
result # в результате нам выдаст начало и конец того, что искали

<_sre.SRE_Match object; span=(47, 51), match='р-не'>

In [ ]:
train.passport_issuer_name = train.passport_issuer_name.str.replace(u'р-(а|й|о|н|е)*',u'район ')
train.passport_issuer_name = train.passport_issuer_name.str.replace(u' г( |\.|(ор(\.| )))', u' город ')
train.passport_issuer_name = train.passport_issuer_name.str.replace(u' р(\.|есп\. )', u' республика ')

Пропишите так же для сокращений административный(адм. админ. администр. итд), округ(окр., округа, окр), и административный округ(ао)

In [ ]:
train.passport_issuer_name = train.passport_issuer_name.str.replace(u' адм[ин\.|инистр\.|инистратив\.] ',u' административный ')
train.passport_issuer_name = train.passport_issuer_name.str.replace(u' окр[уг|уга|\.] ',u'округ ')
train.passport_issuer_name = train.passport_issuer_name.str.replace(u' ао ',u' административный округ ')
## - ВАШ КОД ТУТ - ##

##### Символы

Теперь избавимся от всех лишних символов, кроме русских букв, дефисов и пробелов. Это связано с тем, что паспорт о одинаковым подразделением может выдаваться отделами с разными номерами, и это ухудшит дальнейшую кодировку:

In [ ]:
train.passport_issuer_name = train.passport_issuer_name.str.replace(u' - ?', u'-')
train.passport_issuer_name = train.passport_issuer_name.str.replace(u'[^а-я -]','')
train.passport_issuer_name = train.passport_issuer_name.str.replace(u'- ',' ')

Избавтесь от пробелов так же

In [ ]:
#train.passport_issuer_name = train.passport_issuer_name.str.strip()
train.passport_issuer_name = train.passport_issuer_name.str.replace(u'\s+',' ')
## - ВАШ КОД ТУТ - ##

##### Аббревиатуры

На следующем шаге, надо расшифровать аббревиатуры, типа УВД, УФНС, ЦАО, ВАО и т.д., т.к. этих их в принципе не много, но на качестве дальнейшего кодирования это скажется положительно. Например если у нас будет две записи «УВД» и «управление внутренних дел», то закодированы они будут по разному, т. к. для компьютера это разные значения.
Итак перейдем к расшифровке. И, для начала, заведем словарь сокращений, с помощью которого мы и сделаем расшифровку:

Добавьте в этот словарик еще аббревиатуры юао(южный), юзао(юго-западный), ювао(юго-восточный), пс(паспортный стол), тп(территориальный пункт)

In [ ]:
sokr = {u'нао': u'ненецкий автономный округ',
u'хмао': u'ханты-мансийский автономный округ',
u'чао': u'чукотский автономный округ',
u'янао': u'ямало-ненецкий автономный округ',
u'вао': u'восточный административный округ',
u'цао': u'центральный административный округ',
u'зао': u'западный административный округ',
u'cао': u'северный административный округ',
u'свао': u'северо-восточный округ',
u'сзао': u'северо-западный округ',
u'оуфмс': u'отдел управление федеральной миграционной службы',
u'офмс': u'отдел федеральной миграционной службы',
u'уфмс': u'управление федеральной миграционной службы',
u'увд': u'управление внутренних дел',
u'ровд': u'районный отдел внутренних дел',
u'говд': u'городской отдел внутренних дел',
u'рувд': u'районное управление внутренних дел',
u'овд': u'отдел внутренних дел',
u'оувд': u'отдел управления внутренних дел',
u'мро': u'межрайонный отдел',
u'юао': u'южный',
u'ювао': u'юго-восточный',
u'пс': u'паспортный стол',
u'тп': u'территориальный пункт'}
## - ВАШ КОД ТУТ - ##}

Теперь, собственно произведем расшифровку абривеатур и отформатируем полученные записи:

In [ ]:
for i in sokr.keys():
    train.passport_issuer_name = train.passport_issuer_name.str.replace(u'( %s )|(^%s)|(%s$)' % (i,i,i), u' %s ' % (sokr[i]))
    
#удалим лишние пробелы в конце и начале строки
train.passport_issuer_name = train.passport_issuer_name.str.lstrip()
train.passport_issuer_name = train.passport_issuer_name.str.rstrip()

Нам нужно найти такое k, начиная с которого значение критерия k-means будет убывать не слишком быстро. Этот эффект очень визуально похож на локоть и отсюда, собственно, название этого метода (Метод Локтя). Например, для данных на рисунке выше, таким k будет k равное 4. Важно понимать, что все эти эвристики и меры качества в кластеризации носят лишь рекомендательный характер.

####Столбец - Дата выдачи:

Предварительный этап обработки поля «кем выдан паспорт» на этом закончим. И перейдем к полю, в котором находится дата выдачи.
Как можно заметить данные в нем хранятся в виде: месяцMгод.
Соответственно можно просто убрать букву «M» и привести поле к числовому типу. Но если хорошо подумать, то это поле можно удалить, т.к. на один месяц в году может приходиться несколько подразделений выдававших паспорт, и соответственно это может испортить нашу модель. Исходя из этого удалим его из выборки:

In [ ]:
train = train.drop(['passport_issue_month/year'], axis=1)

Теперь мы можем перейти к анализу данных.

##Анализ данных

Итак, данные для построения модели у нас есть, но они находятся в текстовом виде. Для построения модели хорошо бы было их закодировать в числовом виде.
Авторы пакета scikit-learn заботливо о нас позаботились и добавили несколько способов для извлечения и кодирования текстовых данных.

* [FeatureHasher](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.FeatureHasher.html#sklearn.feature_extraction.FeatureHasher)

* [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer)

* [HashingVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html#sklearn.feature_extraction.text.HashingVectorizer)

**FeatureHasher** преобразовывает строку в числовой массив заданной длинной с помощью хэш-функции (32-разрядная версия Murmurhash3)

**CountVectorizer** преобразовывает входной текст в матрицу, значениями которой, являются количества вхождения данного ключа(слова) в текст. В отличие от FeatureHasher имеет больше настраиваемых параметров(например можно задать токенизатор), но работает медленнее.
Для более точного понимания работы CountVectorizer приведем простой пример. Допустим есть таблица с текстовыми значениями:

раз два три

три четыре два два

раз раз раз четыре


Для начала CountVectorizer собирает уникальные ключи из всех записей, в нашем примере это будет:

[раз, два, три, четыре]

Длина списка из уникальных ключей и будет длиной нашего закодированного текста (в нашем случае это 4). А номера элементов будут соответствовать, количеству раз встречи данного ключа с данным номером в строке:

раз два три --> [1,1,1,0]
три четыре два два --> [0,2,1,1]

Соответственно после кодировки, применения данного метода мы получим:

1,1,1,0

0,2,1,1

3,0,0,1


**HashingVectorizer** является смесью двух выше описанных методов. В нем можно и регулировать размер закодированной строки (как в FeatureHasher) и настраивать токенизатор (как в CountVectorizer). К тому же его производительность ближе к FeatureHasher.




Итак, вернемся к анализу. Если мы посмотрим по внимательнее на наш набор данных то можно заметить, что есть похожие строки но записанные по разному например: "… республика карелия..." и "… по республике карелия...".

Для этой задачи хорошо подходит pymorphy или nltk. Мы будем использовать первый, т.к. он изначально создавался для работы с русским языком. Итак, функция которая будет отвечать за нормализацию и очиску строки выглядит так:

In [ ]:
def f_tokenizer(s):
    path="/usr/local/lib/python3.6/dist-packages/pymorphy2_dicts_ru/data"
    morph = pymorphy2.MorphAnalyzer(path=path, lang='ru')
    if isinstance(s, str):
        t = s.split(' ')
    else:
        t = s
    f = []
    for j in t:
        m = morph.parse(j.replace('.',''))
        if len(m) != 0:
            wrd = m[0]
            if wrd.tag.POS not in ('NUMR','PREP','CONJ','PRCL','INTJ'):
                f.append(wrd.normal_form)
    return f

Функция делает следующее:
* Сначала она преобразовывает строку в список
* Затем для всех слов производит разбор
* Если слово является числительным, предикативном, предлогом, союзом, частицей или междометием не включаем его в конечный набор
* Если слово не попало в предыдущий список, берем его нормальную форму и добавляем в финальный набор

Теперь, когда есть функция для нормализации можно приступить к кодированию с помощью метода CountVectorizer. Он выбран потому, что ему можно передать нашу функцию, как токенизатор и он составит список ключей по значениям полученным в результате работы нашей функции:

In [ ]:
coder = HashingVectorizer(tokenizer=f_tokenizer, n_features=256)

Как можно заметить при создании метода кроме токенизатора мы задаем еще один параметр n_features. Через данный параметр задается длина закодированной строки (в нашем случае строка кодируется при помощи 256 столбцов). Кроме того, у HashingVectorizer есть еще одно преимущество перед CountVectorizer, но сразу может выполнять нормализацию значений, что хорошо для таких алгоритмов, как SVM.
Теперь применим наш кодировщик к обучающему набору:

In [ ]:
TrainNotDuble = train.iloc[1:50000].drop_duplicates() 
# тут мы берем значение от 1 до 10000. Выполняться код в таком случае будет 8:30 минут. 
# Можете взять больше или меньше - ждать придется соответственно, но и работа функции изменится!

In [ ]:
trn = coder.fit_transform(TrainNotDuble.passport_issuer_name.tolist()).toarray()

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


KeyboardInterrupt: ignored

#Построение модели


Для начала нам надо задать значения для столбца, в котором будут содержаться метки классов:


In [ ]:
target = TrainNotDuble.passport_div_code.values

Задача, которую мы решаем сегодня, принадлежит к классу задач классификации со множеством классов. Для решения данной задачи лучше всего подошел алгоритм RandomForest. Остальные алгоритмы показали очень плохие результаты (менее 50%) поэтому я решил не занимать место в статье. При желании любой интересующийся может проверить данные результаты.
Для оценки качества классификации будем использовать количество документов по которым принято правильное решение, т. е.

Accuracy = {P} / {N}

, где P — количество документов по которым классификатор принял правильное решение, а N – размер обучающей выборки.
В пакете scikit-learn для этого есть функция: accuracy_score
Перед началом построения собственно модели, давайте сократим размерность с помощью «метода главных компонент», т.к. 256 столбцов для обучения довольно много:

In [ ]:
pca = PCA(n_components = 15)
trn = pca.fit_transform(trn)

Модель будет выглядеть так:

In [ ]:
model = RandomForestClassifier(n_estimators = 100, criterion='entropy')

TRNtrain, TRNtest, TARtrain, TARtest = train_test_split(trn, target, test_size=0.4)
model.fit(TRNtrain, TARtrain)
print ('accuracy_score: ', accuracy_score(TARtest, model.predict(TRNtest)))

accuracy_score:  0.5209471766848816


#Заключение

Точность полученная в результате работы - 0.5209471766848816
. Эта точность близка к "угадыванию". Для улучшения точности можно обрабытывать грамматические ошибки и дргугие опечатки. Обучение тестовой выборки при выполнении работы не производилось так как оно сводилось бы лишь к приведению выборки к нужному виду.